In [1]:
import json
import pandas as pd

In [2]:
def load_data():
    with open("cabral_full_catalog_with_ratings.json", "r", encoding="utf-8") as f:
        raw = json.load(f)

    rows = []
    for parent_handle, group in raw.items():
        parent_title = group.get("title")
        for sub_handle, sub in group.get("subs", {}).items():
            sub_title = sub.get("title")
            for prod in sub.get("products", []):
                prod_data = {
                    "parent_handle": parent_handle,
                    "collection": parent_title,
                    "sub_handle": sub_handle,
                    "sub_title": sub_title,
                    "title": prod.get("title"),
                    "price": extract_price(prod.get("price")),
                    "sku": prod.get("sku"),
                    "description": prod.get("description"),
                    "url": prod.get("url"),
                    "images": prod.get("images"),
                    "review_count": prod.get("count_reviews"),
                    #"reviews": prod.get("reviews"),
                    "avg_rating": prod.get("average_rating"),

                }
                rows.append(prod_data)
    return pd.DataFrame(rows)

def extract_price(p):
    try:
        return float(p.replace("₹", "").replace(",", ""))
    except:
        return None
    
# --- Title and Callout ---
#st.title("Cabral Outdoors Catalog") 

df = load_data()

df["review_count"] = df["review_count"].fillna(0)
df["review_count"] = df["review_count"].astype(int)

df["avg_rating"] = df["avg_rating"].fillna(0.0)
df["avg_rating"] = df["avg_rating"].astype(float)

# Load Sub_collection_categories.json
with open("Sub_collection_categories.json", "r", encoding="utf-8") as f:
    subcat_json = json.load(f)

# Build mapping: sub_title -> parent title (i.e., sub_collection)
sub_title_to_parent = {}
for category in subcat_json.values():
    parent_title = category["title"]
    for sub_key, sub_title in category["subs"].items():
        sub_title_to_parent[sub_title] = parent_title

# Apply mapping to df['sub_title']
df["sub_collection"] = df["sub_title"].map(sub_title_to_parent).fillna("Unknown")

df = df.drop(['parent_handle','sub_handle'], axis=1)


In [3]:
df

,collection,sub_title,title,price,sku,description,url,images,review_count,avg_rating,sub_collection
0,Fishing,Spinning Reels,Okuma Longbow XT Baitfeeder Spinning Reel | LB...,4750.0,None,Description:-\nOkuma Longbow XT Baitfeeder spi...,https://cabraloutdoors.com/collections/spinnin...,[https://cabraloutdoors.com/cdn/shop/files/ZMA...,0,0.0,Reels
1,Fishing,Spinning Reels,Okuma CDX-65 Coronado Baitfeeder Spinning Reel,7200.0,42295479828598,Description:-\nOkuma Coronado CDX Baitfeeder s...,https://cabraloutdoors.com/collections/spinnin...,[https://cabraloutdoors.com/cdn/shop/files/ZMA...,0,0.0,Reels
2,Fishing,Spinning Reels,Okuma Acuador 6000 Spinning Reel,3250.0,42295471702134,Description:-\nThe newly redesigned Acuador Sp...,https://cabraloutdoors.com/collections/spinnin...,[https://cabraloutdoors.com/cdn/shop/files/ZMA...,0,0.0,Reels
3,Fishing,Spinning Reels,Okuma Safina SW-X SNW-4000A-6000A Spinning Reel,3550.0,42293766094966,Description:-\nThe Safina SW-X is Okuma’s salt...,https://cabraloutdoors.com/collections/spinnin...,[https://cabraloutdoors.com/cdn/shop/files/ZMA...,0,0.0,Reels
4,Fishing,Spinning Reels,Penn Battle® IV 4000DX-6000DX Spinning Reel,10190.0,BTLIV5000DX,\n\n\n\nThe Penn Battle IV DX Spinning Reels a...,https://cabraloutdoors.com/collections/spinnin...,[https://cabraloutdoors.com/cdn/shop/files/399...,0,0.0,Reels
...,...,...,...,...,...,...,...,...,...,...,...
1651,Apparel / Merchandise,Arm Sleeves,Okuma Sun Protector Arm Sleeves | Size: S-M,560.0,39703345528950,﻿Okuma has introduced UPF-rated sun fishing sl...,https://cabraloutdoors.com/collections/arm-sle...,[https://cabraloutdoors.com/cdn/shop/products/...,1,5.0,Apparel / Merchandise
1652,Apparel / Merchandise,Arm Sleeves,BKK Arm Sleeves | Size: M-XL,900.0,39791524511862,﻿BKK Arm Sleeves\nThe BKK Arm Sleeves are aime...,https://cabraloutdoors.com/collections/arm-sle...,[https://cabraloutdoors.com/cdn/shop/products/...,0,0.0,Apparel / Merchandise
1653,Apparel / Merchandise,Arm Sleeves,Pro-Hunter Arm Shield,590.0,40648940355702,\nProtect your arms from the blazing heat whil...,https://cabraloutdoors.com/collections/arm-sle...,[https://cabraloutdoors.com/cdn/shop/files/Pro...,0,0.0,Apparel / Merchandise
1654,Apparel / Merchandise,Arm Sleeves,Duo UV Arm Guard,1060.0,39481456263286,Duo UV Arm Guard\nKeep yourself comfortable an...,https://cabraloutdoors.com/collections/arm-sle...,[https://cabraloutdoors.com/cdn/shop/products/...,0,0.0,Apparel / Merchandise


In [4]:
df.to_csv("Final.csv",index=False)

In [14]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

%matplotlib inline

# Load the dataset
df = pd.read_csv('Final.csv')

# Filter for the 'Fishing' sub-collection
fishing_df = df[df['sub_collection'] == 'Fishing'].copy()

# Data Transformation and Cleaning
# Convert 'price' to a numeric type, removing the currency symbol and handling potential errors.
fishing_df['price'] = fishing_df['price'].replace({'\$': ''}, regex=True).astype(float)

# Convert 'avg_rating' and 'review_count' to numeric, coercing errors to NaN
fishing_df['avg_rating'] = pd.to_numeric(fishing_df['avg_rating'], errors='coerce')
fishing_df['review_count'] = pd.to_numeric(fishing_df['review_count'], errors='coerce')

# Drop rows where avg_rating or review_count could not be converted, as they are essential for our analysis
fishing_df.dropna(subset=['avg_rating', 'review_count'], inplace=True)

# --- KPI Calculations ---
total_products = len(fishing_df)
average_price = fishing_df['price'].mean()
average_rating = fishing_df['avg_rating'].mean()
total_reviews = fishing_df['review_count'].sum()

In [19]:
# --- Dashboard Mockup Creation ---
fig = plt.figure(figsize=(12, 16), facecolor='#f0f0f0')
fig.suptitle('Fishing Collection KPI Dashboard Mockup', fontsize=20, weight='bold', y=0.98)
plt.show()

<Figure size 1200x1600 with 0 Axes>

In [ ]:




# --- KPI Cards ---
gs_kpi = fig.add_gridspec(1, 4, top=0.9, hspace=0.4, wspace=0.6)

def create_kpi_card(ax, title, value, color):
    ax.set_facecolor(color)
    ax.text(0.5, 0.6, title, ha='center', va='center', fontsize=14, weight='bold', color='white')
    ax.text(0.5, 0.3, value, ha='center', va='center', fontsize=20, color='white')
    ax.set_xticks([])
    ax.set_yticks([])
    for spine in ax.spines.values():
        spine.set_edgecolor('white')
        spine.set_linewidth(2)

ax_kpi1 = fig.add_subplot(gs_kpi[0, 0])
create_kpi_card(ax_kpi1, 'Total Products', f'{total_products}', '#4c72b0')

ax_kpi2 = fig.add_subplot(gs_kpi[0, 1])
create_kpi_card(ax_kpi2, 'Avg. Price', f'${average_price:.2f}', '#55a868')

ax_kpi3 = fig.add_subplot(gs_kpi[0, 2])
create_kpi_card(ax_kpi3, 'Avg. Rating', f'{average_rating:.2f}', '#c44e52')

ax_kpi4 = fig.add_subplot(gs_kpi[0, 3])
create_kpi_card(ax_kpi4, 'Total Reviews', f'{int(total_reviews)}', '#8172b2')


# --- Main dashboard layout ---
gs_main = fig.add_gridspec(4, 2, top=0.8, hspace=0.4, wspace=0.3)

# --- Price Distribution ---
ax1 = fig.add_subplot(gs_main[0, 0])
sns.histplot(data=fishing_df, x='price', bins=20, kde=True, ax=ax1, color='#55a868')
ax1.set_title('Price Distribution', fontsize=14, weight='bold')
ax1.set_xlabel('Price ($)')
ax1.set_ylabel('Number of Products')

# --- Rating Distribution ---
ax2 = fig.add_subplot(gs_main[0, 1])
rating_counts = fishing_df['rating'].value_counts().sort_index()
sns.barplot(x=rating_counts.index, y=rating_counts.values, ax=ax2, palette='viridis')
ax2.set_title('Rating Distribution', fontsize=14, weight='bold')
ax2.set_xlabel('Rating')
ax2.set_ylabel('Number of Products')

# --- Top 5 Products by Reviews ---
ax3 = fig.add_subplot(gs_main[1, :])
top_5_by_reviews = fishing_df.nlargest(5, 'reviews_count')[['title', 'reviews_count']]
ax3.axis('off')
ax3.set_title('Top 5 Products by Number of Reviews', fontsize=14, weight='bold', loc='left')
table = ax3.table(cellText=top_5_by_reviews.values,
                  colLabels=top_5_by_reviews.columns,
                  cellLoc='center',
                  loc='center',
                  colWidths=[0.8, 0.2])
table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1, 1.5)

# --- Top 5 Products by Rating ---
ax4 = fig.add_subplot(gs_main[2, :])
# Consider products with a minimum number of reviews to make the rating more reliable
top_5_by_rating = fishing_df[fishing_df['reviews_count'] >= 5].nlargest(5, 'rating')[['title', 'rating', 'reviews_count']]
ax4.axis('off')
ax4.set_title('Top 5 Products by Rating (with >= 5 reviews)', fontsize=14, weight='bold', loc='left')
table = ax4.table(cellText=top_5_by_rating.values,
                  colLabels=top_5_by_rating.columns,
                  cellLoc='center',
                  loc='center',
                  colWidths=[0.7, 0.15, 0.15])
table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1, 1.5)

plt.savefig('kpi_dashboard_mockup.png', bbox_inches='tight', pad_inches=0.5, facecolor='#f0f0f0')
plt.close()

# --- Text Analysis ---
from wordcloud import WordCloud

# Word Cloud from Product Titles
text_for_wordcloud = ' '.join(fishing_df['title'].dropna())
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text_for_wordcloud)

plt.figure(figsize=(15, 7))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud from Product Titles (Fishing Collection)', fontsize=20)
plt.savefig('fishing_title_wordcloud.png', bbox_inches='tight')
plt.close()

# Word Cloud from Reviews
# Assuming 'review' column contains review text. If not, this part will be skipped.
if 'review' in fishing_df.columns and not fishing_df['review'].isnull().all():
    review_text = ' '.join(fishing_df['review'].dropna())
    if review_text:
        wordcloud_reviews = WordCloud(width=800, height=400, background_color='white').generate(review_text)
        plt.figure(figsize=(15, 7))
        plt.imshow(wordcloud_reviews, interpolation='bilinear')
        plt.axis('off')
        plt.title('Word Cloud from Reviews (Fishing Collection)', fontsize=20)
        plt.savefig('fishing_reviews_wordcloud.png', bbox_inches='tight')
        plt.close()

# Create a filtered CSV for the user
#fishing_df.to_csv('fishing_products.csv', index=False)

<Figure size 1200x1600 with 0 Axes>

KeyError: 'rating'